<a href="https://colab.research.google.com/github/aiyoheidei/ca4/blob/main/ca4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision

In [1]:
!pip3 install captcha

     |████████████████████████████████| 102 kB 15.1 MB/s 


In [ ]:
import cv2
from captcha.image import ImageCaptcha
import numpy as np
from PIL import Image
import random
import os
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import time


number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            'V', 'W', 'X', 'Y', 'Z']

width = 80
height = 40

captcha_size = 2
char_set = number + alphabet + ALPHABET


def random_captcha_text():
    captcha_text = []
    for i in range(captcha_size):
        c = random.choice(char_set)
        captcha_text.append(c)
    return captcha_text


def gen_captcha_text_and_image(total_number, filename):
    r = 0.6
    image = ImageCaptcha(width=width, height=height, font_sizes=(int(42 * r), int(50 * r), int(56 * r)))
    for i in range(total_number):
        captcha_text = random_captcha_text()
        captcha_text = ''.join(captcha_text)
        captcha = image.generate(captcha_text)
        image.write(captcha_text,'./images/' + filename + '/' + captcha_text + '_' + str(time.time()) + '.jpg')


def one_hot(text):
    # vector = np.zeros(4*62)  #(10+26+26)*4
    vector = np.zeros(captcha_size * 62)  # (10+26+26)*4

    def char2pos(c):
        if c == '_':
            k = 62
            return k
        k = ord(c) - 48
        if k > 9:
            k = ord(c) - 55
            if k > 35:
                k = ord(c) - 61
                if k > 61:
                    raise ValueError('No Map')
        return k

    for i, c in enumerate(text):
        idx = i * 62 + char2pos(c)
        vector[idx] = 1
    return vector


class MyDataSet(Dataset):
    def __init__(self, dir):
        self.dir = dir
        self.img_name = next(os.walk(self.dir))[2]

    def __getitem__(self, index):
        img_path = os.path.join(self.dir, self.img_name[index])
        img = cv2.imread(img_path, 0)
        img = img / 255.
        img = torch.from_numpy(img).float()
        img = torch.unsqueeze(img, 0)
        # label = torch.from_numpy(one_hot(self.img_name[index][:-4])).float()
        label = torch.from_numpy(one_hot(self.img_name[index][:captcha_size])).float()
        # print(self.img_name[index][:captcha_size])
        return img, label

    def __len__(self):
        return len(self.img_name)


class CNN_Network(nn.Module):
    def __init__(self):
        super(CNN_Network, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, stride=1, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, stride=1, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(stride=2, kernel_size=2),  # 30 80     20 60
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, stride=1, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 15 40       10 30
        )

        self.fc = nn.Sequential(
            # nn.Linear(128 * 15 * 40, 2048),
            # nn.Linear(128 * 10 * 30, 2048),
            nn.Linear(25600, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 124)
            # nn.Linear(1024, 62)
        )

    def forward(self, x):
        x = self.layer1(x)
        # print("layer1:", x.shape)
        x = self.layer2(x)
        # print("layer2:", x.shape)
        x = self.layer3(x)
        # print("layer3:", x.shape)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


def train(net, train_iter, test_iter, optimizer, criteon, device, num_epochs):
    # global_step = 0
    net = net.to(device)
    test_acc_list = []
    flag = 0.0
    for epoch in range(num_epochs):
        for idx, (x, y) in enumerate(train_iter):
            x = x.to(device)
            y = y.to(device)
            y_hat = net(x)
            loss = criteon(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # global_step += 1
            # viz.line([loss.item()], [global_step], win='train_loss', update='append')
        print("loss: ", loss.item())
        test_acc = get_acc(net, test_iter, device)
        print("test_acc=", test_acc)
        if test_acc > flag:
            torch.save(net.state_dict(), "yanzhengma.pkl")
            flag = test_acc
        test_acc_list.append(test_acc)

        # viz.line([test_acc], [epoch], win='test_acc', update='append')


def get_acc(net, data_iter, device):
    acc_sum, n = 0, 0
    for x, y in data_iter:
        x = x.to(device)
        y = y.to(device)
        y_hat = net(x)
        pre1 = torch.argmax(y_hat[:, :62], dim=1)
        real1 = torch.argmax(y[:, :62], dim=1)
        pre2 = torch.argmax(y_hat[:, 62:124], dim=1)
        real2 = torch.argmax(y[:, 62:124], dim=1)
        # pre3 = torch.argmax(y_hat[:, 124:186], dim=1)
        # real3 = torch.argmax(y[:, 124:186], dim=1)
        # pre4 = torch.argmax(y_hat[:, 186:], dim=1)
        # real4 = torch.argmax(y[:, 186:], dim=1)
        # pre_lable = torch.cat((pre1, pre2, pre3, pre4), 0).view(4, -1)
        pre_lable = torch.cat((pre1, pre2), 0).view(2, -1)
        # real_label = torch.cat((real1, real2, real3, real4), 0).view(4, -1)
        real_label = torch.cat((real1, real2), 0).view(2, -1)
        bool_ = (pre_lable == real_label).transpose(0, 1)
        n += y.shape[0]
        for i in range(0, y.shape[0]):
            if bool_[i].int().sum().item() == captcha_size:
                acc_sum += 1
    return acc_sum / n



if __name__ == "__main__":

    # import visdom

    # viz = visdom.Visdom()
    # viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
    # # viz.line([0.], [0.], win='train_acc', opts=dict(title='train acc'))
    # viz.line([0.], [0.], win='test_acc', opts=dict(title='test acc'))

    if not os.path.exists(r'./images/train'):
        os.makedirs(r'./images/train')
        gen_captcha_text_and_image(15000, 'train')
    if not os.path.exists(r'./images/test'):
        os.makedirs(r'./images/test')
        gen_captcha_text_and_image(1000, 'test')

    epoch = 2000
    # epoch=3
    batch_size = 500
    # batch_size=100
    lr = 0.001

    train_dataset = MyDataSet('./images/train')
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    test_dataset = MyDataSet('./images/test')
    test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

    net = CNN_Network()
    if os.path.exists("yanzhengma.pkl"):
        net.load_state_dict(torch.load("yanzhengma.pkl"))
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    criteon = nn.MultiLabelSoftMarginLoss()
    # criteon = nn.CrossEntropyLoss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    train(net, train_loader, test_loader, optimizer, criteon, device, epoch)


loss:  0.0879615917801857
test_acc= 0.0
loss:  0.08290625363588333
test_acc= 0.0
loss:  0.08264827728271484
test_acc= 0.0
loss:  0.08271542936563492
test_acc= 0.001
loss:  0.08260007947683334
test_acc= 0.0
loss:  0.08262622356414795
test_acc= 0.0
loss:  0.08269357681274414
test_acc= 0.001
loss:  0.08268333971500397
test_acc= 0.0
loss:  0.08265307545661926
test_acc= 0.0
loss:  0.08271036297082901
test_acc= 0.0
loss:  0.08269393444061279
test_acc= 0.001
loss:  0.08272109180688858
test_acc= 0.0
loss:  0.08263443410396576
test_acc= 0.0
loss:  0.08270323276519775
test_acc= 0.0
loss:  0.08287303894758224
test_acc= 0.001


KeyboardInterrupt: ignored

In [2]:
import cv2
from captcha.image import ImageCaptcha
import numpy as np
from PIL import Image
import random
import os
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import time


number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            'V', 'W', 'X', 'Y', 'Z']

width = 120
height = 40

captcha_size = 4
char_set = number + alphabet + ALPHABET


def random_captcha_text():
    captcha_text = []
    for i in range(captcha_size):
        c = random.choice(char_set)
        captcha_text.append(c)
    return captcha_text


def gen_captcha_text_and_image(total_number, filename):
    r = 0.6
    image = ImageCaptcha(width=width, height=height, font_sizes=(int(42 * r), int(50 * r), int(56 * r)))
    for i in range(total_number):
        captcha_text = random_captcha_text()
        captcha_text = ''.join(captcha_text)
        captcha = image.generate(captcha_text)
        image.write(captcha_text,'./images/' + filename + '/' + captcha_text + '_' + str(time.time()) + '.jpg')


def one_hot(text):
    # vector = np.zeros(4*62)  #(10+26+26)*4
    vector = np.zeros(captcha_size * 62)  # (10+26+26)*4

    def char2pos(c):
        if c == '_':
            k = 62
            return k
        k = ord(c) - 48
        if k > 9:
            k = ord(c) - 55
            if k > 35:
                k = ord(c) - 61
                if k > 61:
                    raise ValueError('No Map')
        return k

    for i, c in enumerate(text):
        idx = i * 62 + char2pos(c)
        vector[idx] = 1
    return vector


class MyDataSet(Dataset):
    def __init__(self, dir):
        self.dir = dir
        self.img_name = next(os.walk(self.dir))[2]

    def __getitem__(self, index):
        img_path = os.path.join(self.dir, self.img_name[index])
        img = cv2.imread(img_path, 0)
        img = img / 255.
        img = torch.from_numpy(img).float()
        img = torch.unsqueeze(img, 0)
        # label = torch.from_numpy(one_hot(self.img_name[index][:-4])).float()
        label = torch.from_numpy(one_hot(self.img_name[index][:captcha_size])).float()
        # print(self.img_name[index][:captcha_size])
        return img, label

    def __len__(self):
        return len(self.img_name)


class CNN_Network(nn.Module):
    def __init__(self):
        super(CNN_Network, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, stride=1, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, stride=1, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(stride=2, kernel_size=2),  # 30 80     20 60
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, stride=1, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 15 40       10 30
        )

        self.fc = nn.Sequential(
            # nn.Linear(128 * 15 * 40, 2048),
            nn.Linear(128 * 10 * 30, 2048),
            # nn.Linear(12800, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 248)
            # nn.Linear(1024, 62)
        )

    def forward(self, x):
        x = self.layer1(x)
        # print("layer1:", x.shape)
        x = self.layer2(x)
        # print("layer2:", x.shape)
        x = self.layer3(x)
        # print("layer3:", x.shape)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


def train(net, train_iter, test_iter, optimizer, criteon, device, num_epochs):
    # global_step = 0
    net = net.to(device)
    train_acc_list, test_acc_list, train_loss_list, test_loss_list = [], [], [], []
    flag = 0.0
    for epoch in range(num_epochs):
        # train_loss,n=0.0,0
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            loss = criteon(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # global_step += 1
            # viz.line([loss.item()], [global_step], win='train_loss', update='append')
        print("epoch:", epoch)
        print("loss: ", loss.item())
        # train_acc = get_acc(net,train_iter,device)
        test_acc = get_acc(net, test_iter, device)
        print("test_acc=", test_acc)
        print()
        if test_acc > flag:
            torch.save(net.state_dict(), "yanzhengma.pkl")
            flag = test_acc
        # train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        # viz.line([train_acc], [epoch], win='train_acc', update='append')
        # viz.line([test_acc], [epoch], win='test_acc', update='append')

        # train_loss_list.append(train_loss)

    # draw_losss(train_loss_list,test_loss_list,num_epochs)
    draw_acc(train_acc_list,test_acc_list,num_epochs)


def get_acc(net, data_iter, device):
    acc_sum, n = 0, 0
    for x, y in data_iter:
        x = x.to(device)
        y = y.to(device)
        y_hat = net(x)
        pre1 = torch.argmax(y_hat[:, :62], dim=1)
        real1 = torch.argmax(y[:, :62], dim=1)
        pre2 = torch.argmax(y_hat[:, 62:124], dim=1)
        real2 = torch.argmax(y[:, 62:124], dim=1)
        pre3 = torch.argmax(y_hat[:, 124:186], dim=1)
        real3 = torch.argmax(y[:, 124:186], dim=1)
        pre4 = torch.argmax(y_hat[:, 186:], dim=1)
        real4 = torch.argmax(y[:, 186:], dim=1)
        pre_lable = torch.cat((pre1, pre2, pre3, pre4), 0).view(4, -1)
        real_label = torch.cat((real1, real2, real3, real4), 0).view(4, -1)
        bool_ = (pre_lable == real_label).transpose(0, 1)
        n += y.shape[0]
        for i in range(0, y.shape[0]):
            if bool_[i].int().sum().item() == captcha_size:
                acc_sum += 1
    return acc_sum / n


def draw_losss(train_loss, test_loss, epoch):
    plt.clf()
    x = [i for i in range(epoch)]
    plt.plot(x, train_loss, label='train_loss')
    plt.plot(x, test_loss, label='test_loss')
    plt.legend()
    plt.title("loss goes by epoch")
    plt.xlabel('eopch')
    plt.ylabel('loss_value')
    plt.savefig('loss.png')


def draw_acc(train_acc, test_acc, epoch):
    plt.clf()
    x = [i for i in range(epoch)]
    plt.plot(x, train_acc, label='train_acc')
    plt.plot(x, test_acc, label='test_acc')
    plt.legend()
    plt.title("acc goes by epoch")
    plt.xlabel('eopch')
    plt.ylabel('acc_value')
    plt.savefig('acc.png')


if __name__ == "__main__":

    # import visdom

    # viz = visdom.Visdom()
    # viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
    # # viz.line([0.], [0.], win='train_acc', opts=dict(title='train acc'))
    # viz.line([0.], [0.], win='test_acc', opts=dict(title='test acc'))

    if not os.path.exists(r'./images/train'):
        os.makedirs(r'./images/train')
        gen_captcha_text_and_image(100000, 'train')
    if not os.path.exists(r'./images/test'):
        os.makedirs(r'./images/test')
        gen_captcha_text_and_image(5000, 'test')

    epoch = 200
    # epoch=3
    batch_size = 64
    # batch_size=100
    lr = 0.001

    train_dataset = MyDataSet('./images/train')
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    test_dataset = MyDataSet('./images/test')
    test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

    net = CNN_Network()
    if os.path.exists("yanzhengma.pkl"):
        net.load_state_dict(torch.load("yanzhengma.pkl"))
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    criteon = nn.MultiLabelSoftMarginLoss()
    # criteon = nn.CrossEntropyLoss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    train(net, train_loader, test_loader, optimizer, criteon, device, epoch)


epoch: 0
loss:  0.05627579241991043
test_acc= 0.0068

epoch: 1
loss:  0.035212934017181396
test_acc= 0.118

epoch: 2
loss:  0.021049916744232178
test_acc= 0.3164

epoch: 3
loss:  0.015571873635053635
test_acc= 0.4222

epoch: 4
loss:  0.009311707690358162
test_acc= 0.5134

epoch: 5
loss:  0.013146892189979553
test_acc= 0.5544

epoch: 6
loss:  0.011184454895555973
test_acc= 0.571

epoch: 7
loss:  0.009415711276233196
test_acc= 0.5998

epoch: 8
loss:  0.006388316862285137
test_acc= 0.6094

epoch: 9
loss:  0.006816689856350422
test_acc= 0.616

epoch: 10
loss:  0.006296129431575537
test_acc= 0.632

epoch: 11
loss:  0.0032184047158807516
test_acc= 0.6218

epoch: 12
loss:  0.004412776790559292
test_acc= 0.6298

epoch: 13
loss:  0.001708436873741448
test_acc= 0.6216

epoch: 14
loss:  0.0023579364642500877
test_acc= 0.62

epoch: 15
loss:  0.00206652469933033
test_acc= 0.625

epoch: 16
loss:  0.0030587604269385338
test_acc= 0.6252

epoch: 17
loss:  0.0022391059901565313
test_acc= 0.6294

epoch: 

KeyboardInterrupt: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.is_available()

True